# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import string
import nltk
import pickle
import json
sys.path.append('C:/Users/User/Desktop/r8')
import tensorflow as tf
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
from collections import Counter
from data_preprocessing import *

C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\frame

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-train-stemmed.csv')
valid_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-dev-stemmed.csv')
test_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-test-stemmed.csv')
train_df.head()

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


In [3]:
# train_df와 valid_df 결합
train_val_df = pd.concat([train_df, valid_df])

# 전처리

In [4]:
train_val_df['text']=train_val_df['text'].apply(clean_text)
test_df['text']=test_df['text'].apply(clean_text)

In [5]:
x_train=list(train_val_df['text'])
y_train=list(train_val_df['intent'])
x_test=list(test_df['text'])
y_test=list(test_df['intent'])

In [6]:
to_txt=x_train+x_test

# 사전 학습 ELMo 불러오기

In [7]:
#사전 학습된 모델이 있는 디렉토리
datadir = 'elmo_model(r8_256)'
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
batcher = Batcher(vocab_file, 50)
 
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
context_embeddings_op = bilm(context_character_ids)
 
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

#테스트 임베딩
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [8]:
with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())     
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [9]:
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  23.36 
Dress-Fashion =  18.2


Cosine Distance Comparison - 

Dress-Technology =  0.83 
Dress-Fashion =  0.87


# 문서 단위 임베딩

In [10]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_lst=[]

In [11]:
for i in range(len(x_train)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./elmo_embedding/train(r8_256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 6, 256)
Shape of generated embeddings =  (1, 176, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  (1, 315, 256)
Shape of generated embeddings =  (1, 241, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 11, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 146, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 86, 256)
Shape of generated embeddings =  (1, 124, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 205, 256)
Shape of generated embeddings 

Shape of generated embeddings =  (1, 181, 256)
Shape of generated embeddings =  (1, 6, 256)
Shape of generated embeddings =  (1, 140, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 76, 256)
Shape of generated embeddings =  (1, 113, 256)
Shape of generated embeddings =  (1, 145, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings 

Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 381, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 455, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 88, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 157, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 265, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =

Shape of generated embeddings =  (1, 96, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 270, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 104, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings = 

Shape of generated embeddings =  (1, 70, 256)
Shape of generated embeddings =  (1, 88, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 12, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 159, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 71, 256)
Shape of generated embeddings =  (1, 69, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 321, 256)
Shape of generated embeddings =  (1, 449, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 11, 256)
Shape of generated embeddings =  (1, 263, 256)
Shape of generated embeddings =  (1, 428, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 400, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 69, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings

Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 75, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 113, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 365, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 203, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =

Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 102, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 303, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 185, 256)
Shape of generated embeddings =  (1, 115, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  (1, 383, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 344, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 349, 256)
Shape of generated embeddings =  (1, 140, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddin

Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 89, 256)
Shape of generated embeddings =  (1, 124, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 105, 256)
Shape of generated embeddings =  (1, 238, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 6, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings 

In [12]:
except_lst

[]

In [10]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_test_lst=[]

In [11]:
for i in range(len(x_test)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./elmo_embedding/test(r8_256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 427, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of generated embeddings =  (1, 372, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 70, 256)
Shape of generated embeddings =  (1, 102, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 167, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 100, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 127, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 111, 256)
Shape of generated embeddings =  (1, 146, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embe

Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 11, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 82, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 133, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 125, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 181, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =

Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 82, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 305, 256)
Shape of generated embeddings =  (1, 109, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 53, 256)
Shape of generated embeddings = 

Shape of generated embeddings =  (1, 194, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 68, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 211, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 162, 256)
Shape of generated embeddings =  (1, 219, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings

Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 91, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 359, 256)
Shape of generated embeddings =  (1, 155, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings = 

...

Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 53, 256)
Shape of generated embeddings =  (1, 129, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (

Shape of generated embeddings =  (1, 14, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 300, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 354, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 256, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  

Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 18, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 84, 256)
Shape of generated embeddings =  (1, 8, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 13, 256)
Shape of generated embeddings =  (1, 132, 256)
Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 69, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 101, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 56, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  

Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 37, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 51, 256)
Shape of generated embeddings =  (1, 24, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 158, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  

Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 7, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 61, 256)
Shape of generated embeddings =  (1, 42, 256)
Shape of generated embeddings =  (1, 15, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1,

In [12]:
except_test_lst

[]